<a href="https://colab.research.google.com/github/NichBry25/BitcoinPricePrediction/blob/main/ModelAgnosticStatisticAggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp

In [2]:
files = [
    "lstm_statistical_results.csv",
    "gru_statistical_results.csv",
    "xgboost_statistical_results.csv",
    "tcn_statistical_results.csv",
    "deepvar_statistical_results.csv"
]

df_all = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)
df_all

,model,wmae_no_macro,wmae_macro,delta_wmae,std_err_no_macro,std_err_macro
0,LSTM,2074.795560,1712.510483,-362.285077,2243.739247,2109.814449
1,GRU,1529.472279,1524.435526,-5.036753,2054.464111,2053.630201
2,XGBoost,1444.833081,1444.833081,0.000000,1617.404854,1617.404854
3,TCN,1627.600038,6633.696282,5006.096244,2134.931433,3890.875713
4,DeepVAR,1520.722896,1585.007499,64.284603,2052.728566,2102.301519


In [4]:
summary_table = df_all[
    ["model", "wmae_no_macro", "wmae_macro", "delta_wmae"]
]

summary_table

,model,wmae_no_macro,wmae_macro,delta_wmae
0,LSTM,2074.795560,1712.510483,-362.285077
1,GRU,1529.472279,1524.435526,-5.036753
2,XGBoost,1444.833081,1444.833081,0.000000
3,TCN,1627.600038,6633.696282,5006.096244
4,DeepVAR,1520.722896,1585.007499,64.284603


In [6]:
summary_table.to_csv("model_agnostic_wmae_summary.csv", index=False)

In [7]:
delta = df_all["delta_wmae"].values

aggregate_stats = {
    "mean_delta_wmae": np.mean(delta),
    "median_delta_wmae": np.median(delta),
    "std_delta_wmae": np.std(delta),
    "min_delta_wmae": np.min(delta),
    "max_delta_wmae": np.max(delta)
}

aggregate_stats

{'mean_delta_wmae': np.float64(940.6118032444394),
 'median_delta_wmae': np.float64(0.0),
 'std_delta_wmae': np.float64(2038.2666722039435),
 'min_delta_wmae': np.float64(-362.2850773268401),
 'max_delta_wmae': np.float64(5006.096243655549)}